# Multiple Factor Analysis (MFA) with categorical variables

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

In [ ]:
import pyreadr
import pandas as pd
result = pyreadr.read_r('./data/poison.rda') # also works for Rds, rda

# done! let's see what we got
# result is a dictionary where keys are the name of objects and the values python
# objects
print(result.keys()) # let's check what objects we got

In [ ]:
# Chargement des données
poison = result["poison"]
poison.head(6)

In [ ]:
poison.info()

In [ ]:
poison2 = pd.DataFrame(data=poison.values,
                     columns = pd.MultiIndex.from_tuples(
    [
        ("desc","Age"),
        ("desc","Time"),
        ("desc2","Sick"),
        ("desc2","Sex"),
        ("symptom","Nausea"),
        ("symptom","Vomiting"),
        ("symptom","Abdominals"),
        ("symptom","Fever"),
        ("symptom","Diarrhae"),
        ("eat","Potato"),
        ("eat","Fish"),
        ("eat","Mayo"),
        ("eat","Courgette"),
        ("eat","Cheese"),
        ("eat","Icecream") 
    ]
))
poison2.index= poison.index

In [ ]:
group = poison2.columns.levels[0].drop(["desc","desc2"]).tolist()
group

In [ ]:
group_sup = poison2.columns.levels[0].drop(group).tolist()
group_sup

In [ ]:
poison2["desc"] = poison2["desc"].astype("float")

In [ ]:
from scientisttools.decomposition import MFA

res_mfa = MFA(n_components=None,
              group=group,
              group_sup=group_sup,
              row_labels=poison2.index,
              parallelize=True)
# Instanciation
res_mfa.fit(poison2)

## Eigenvalues

In [ ]:
from scientisttools.extractfactor import get_eig
eig = get_eig(res_mfa)
eig

## Separate analyses

In [ ]:
res_mfa.separate_analyses_.keys()

## Individuals informations

In [ ]:
from scientisttools.extractfactor import get_mfa_ind
ind = get_mfa_ind(res_mfa)
ind.keys()

### Coordinates

In [ ]:
ind["coord"].head(6)

### Cos2

In [ ]:
ind["cos2"].head(6)

### Contributions

In [ ]:
ind["contrib"].head(6)

### Partiel coordinates

In [ ]:
ind["coord_partiel"].head(6)

### Within inertia

In [ ]:
ind["within_inertia"].head(6)

### Within partial inertia

In [ ]:
ind["within_partial_inertia"].head(6).round(6)

In [ ]:
res_mfa.group_sup_

## Categories

In [ ]:
from scientisttools.extractfactor import get_mfa_var
quali_var = get_mfa_var(res_mfa,element="quali_var")
quali_var.keys()

### Coordinates

In [ ]:
quali_var["coord"]

### Cos2

In [ ]:
quali_var["cos2"]

### Contributions

In [ ]:
# A vérifier
quali_var["contrib"]

### VTest

In [ ]:
quali_var["vtest"]

### Coord partial

In [ ]:
quali_var["coord_partiel"]

### whithin inertia

In [ ]:
quali_var["within_inertia"] # A vérifier

### Whitin partial inertia

In [ ]:
quali_var["within_partial_inertia"] # A vérifier

## Groups

In [ ]:
group = get_mfa_var(res_mfa,element="group")
group.keys()

### Coordinates

In [ ]:
group["coord"].head(6)

### Contributions

In [ ]:
group["contrib"].head(6)

### Correlation

In [ ]:
group["correlation"].head(6)

### Lg

In [ ]:
group["Lg"]

### RV

In [ ]:
group["RV"]

### Dist

In [ ]:
group["dist"]

### Cos2

In [ ]:
group["cos2"]

### Supplementary groups

In [ ]:
group_sup = group["sup"]
group_sup.keys()

#### Coordinates

In [ ]:
group_sup["coord"]

#### Dist2

In [ ]:
group_sup["dist"]

#### Cos2

In [ ]:
group_sup["cos2"]

## Partial axes

In [ ]:
from scientisttools.extractfactor import get_mfa_partial_axes
partial_axes = get_mfa_partial_axes(res_mfa)
partial_axes.keys()

### Coordinates

In [ ]:
partial_axes["coord"]

### Cor

In [ ]:
partial_axes["cor"]

### Corr between

In [ ]:
partial_axes["cor_between"].round(5)

## Inertia ratio

In [ ]:
res_mfa.inertia_ratio_

## Inertia ratio

In [ ]:
res_mfa.inertia_ratio_

## Supplementary qualitatives variables

In [ ]:
quali_var_sup = quali_var["sup"]
quali_var_sup.keys()

### Coordinates

In [ ]:
quali_var_sup["coord"].head(6)

### Cos2

In [ ]:
quali_var_sup["cos2"].head(6)

### VTest

In [ ]:
quali_var_sup["vtest"].head(6)

### Coord partiel

In [ ]:
quali_var_sup["coord_partiel"].head(6)

## Supplementary continues columns

In [ ]:
quanti_var_sup =get_mfa_var(res_mfa,element="quanti_var")["sup"]
quanti_var_sup.keys()

#### Coordinates

In [ ]:
quanti_var_sup["coord"].head(6)

### Cor

In [ ]:
quanti_var_sup["cor"].head(6)

### Cos2

In [ ]:
quanti_var_sup["cos2"].head(6)

## Summary

In [ ]:
res_mfa.summary_quanti_

In [ ]:
res_mfa.summary_quali_

In [ ]:
X = res_mfa.active_data_
res_mfa.transform(X)